In [1]:
import pandas as pd
import wikidataintegrator as wdi

## Prediction of drug-target interactions and drug repositioning via network-based inference (PMID:22589709)

In [4]:
path = '/home/sebastian/jupyter-notebooks/paper_data/Cheng et al 2012 PMID:22589709/DrugBank_Global_Database/Global_Target_List_With_Score_1.xls.xlsx'

pred_data = pd.read_excel(path, header=None)


In [5]:
pred_data.head()

,0,1,2,3,4,5,6,7,8,9,...,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923
0,"Chlorophyll a-b binding protein, chloroplast ...",DB03001,DB02686,DB00819,DB03796,DB01159,DB04079,DB03381,DB02944,DB02687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.277778,0.25,0.125,0.0271673,0.0271673,0.0192308,0.0192308,0.0192308,0.0192308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RNASE4 protein ...,DB02379,DB04184,DB03014,DB01867,DB02061,DB03389,DB02944,DB02212,DB03754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.0417496,0.0402842,0.0279834,0.0188596,0.0187729,0.0179487,0.0175415,0.0129892,0.0129049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Thiazole biosynthetic enzyme, chloroplast ...",DB04147,DB04522,DB02451,DB03381,DB02944,DB02687,DB03796,DB01728,DB02482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pred_data.columns

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923],
           dtype='int64', length=1924)

In [15]:
pred_map = {}

for count, x in pred_data.iterrows():
    
    if pd.notnull(x[0]) and any([pd.notnull(z) for z in x[1:]]):
        drugs = []
        for y in x[1:]:
            if pd.notnull(y):
                drugs.append(y)
        
        pred_map.update({x[0]: drugs})
        
            
            
        

In [16]:
print(len(pred_map))

3487


In [26]:
import requests
import pprint

s = requests.Session()


def sparql_search(items, data_class, category):
    query = '''
    SELECT DISTINCT * WHERE {{
        VALUES ?c {{ wd:{0} }}
        ?c wdt:{2} wd:{1} .
        ?c wdt:P703 wd:Q15978631 .
    }}
    
    '''.format(' wd:'.join(items), data_class, category)
    
    results = wdi.wdi_core.WDItemEngine.execute_sparql_query(query)
    
    
    result_qids = []
    for x in results['results']['bindings']:
        result_qids.append(x['c']['value'].split('/').pop())
    

def search_wikidata(search_term, data_class, category):
    # data class e.g chemical compound (Q11173), category e.g. P31 (instance of), P279 (subclass of)
    
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbsearchentities',
        'search': search_term,
        'language': 'en',
        'type': 'item',
        'format': 'json',
        'limit': 50
    }
    
    r = s.get(url=url, params=params).json()
    if len(r['search']) == 0:
        print('no results for:', search_term)
        
    else:
        items = list()
        for x in r['search']:
            items.append(x['title'])
        
        typefiltered_items = sparql_search(items, data_class, category)
    
    return typefiltered_items

something_found_count = 0
for c, x in enumerate(pred_map):
    sres_length = search_wikidata(x, 'Q8054', 'P279')
    
    if sres_length > 0:
        something_found_count += 1
    
    if c % 100 == 0:
        print('current count:', something_found_count, 'total searches:', c)
    
#     if c > 4:
#         break
    
    
print(something_found_count)
    

not results for: Cytochrome b2, mitochondrial                                                                                                        
current count: 0 total searches: 0
not results for: Hepatic peroxysomal alanine:glyoxylate aminotransferase                                                                             
not results for: Proline oxidase, mitochondrial                                                                                                      
not results for: P-selectin cytoplasmic tail-associated protein                                                                                      
not results for: Cytochrome P450 3A4                                                                                                                 
not results for: Alpha-neurotoxin TX12                                                                                                               
not results for: High affinity immunoglobulin epsilon receptor su